# Import packages

In [1]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd

# Load data:

In [2]:
province_name_mapping = {
    "Alava": "Araba/Álava",
    "Álava": "Araba/Álava",
    "PAÍS VASCO": "País Vasco/Euskadi",
    "PAIS VASCO": "País Vasco/Euskadi",
    "NAVARRA": "Navarra",
    "LA RIOJA": "La Rioja",
    "ARAGÓN": "Aragón",
    "ARAGON": "Aragón",
    "CATALUÑA": "Cataluña/Catalunya",
    "BALEARES": "Illes Balears",
    "Avila": "Ávila",
    "CASTILLA Y LEÓN": "Castilla y León",
    "CASTILLA Y LEON": "Castilla y León",
    "MADRID": "Madrid",
    "CASTILLA-LA MANCHA": "Castilla-La Mancha",
    "C. VALENCIANA": "Comunitat Valenciana",
    "R. DE MURCIA": "Murcia",
    "EXTREMADURA": "Extremadura",
    "ANDALUCÍA": "Andalucía",
    "ANDALUCIA": "Andalucía",
    "CANARIAS": "Canarias",
    "Guipúzcoa": "Gipuzkoa",
    "GALICIA": "Galicia",
    "P. DE ASTURIAS": "Asturias",
    "CANTABRIA": "Cantabria",
    "Vizcaya": "Bizkaia",
    "S.C. de Tenerife": "Santa Cruz de Tenerife",
    "Alicante": "Alacant/Alicante",
    "Castellón": "Castelló/Castellón",
    "Valencia": "València/Valencia",
}

In [3]:
def normalize_province_name(input_province_name: str) -> str:
    if input_province_name in province_name_mapping.keys():
        return province_name_mapping.get(input_province_name)
    return input_province_name

In [4]:
def read_excel_file(file_path):
    df = pd.read_excel(file_path, skiprows=6, na_values="–", header=None)

    # change column names
    df.columns = ["province", "total", "table_olive", "olive_oil"]

    # add year column
    df["year"] = int(file_path.stem)

    # normalize values in province column
    df["province"] = df["province"].str.strip().apply(normalize_province_name)

    # fill NA values in numeric columns with zero
    df[["total", "table_olive", "olive_oil"]] = df[
        ["total", "table_olive", "olive_oil"]
    ].fillna(0)

    # drop empty rows
    df = df.dropna()

    # fix dtype
    df = df.astype(
        {
            "province": "string[pyarrow]",
            "total": "int32",
            "table_olive": "int32",
            "olive_oil": "int32",
        }
    )

    return df

In [5]:
olive_yield_dir = Path("../data/raw/olive_yield/")

In [6]:
df_list = [read_excel_file(file_path) for file_path in olive_yield_dir.iterdir()]

In [7]:
olive_yield_data = pd.concat(df_list, ignore_index=True)

In [8]:
olive_yield_data.head()

,province,total,table_olive,olive_oil,year
0,Araba/Álava,205,0,205,1998
1,País Vasco/Euskadi,205,0,205,1998
2,Navarra,6843,105,6738,1998
3,La Rioja,2868,15,2853,1998
4,Huesca,5727,0,5727,1998


In [9]:
olive_yield_data.groupby(by="province").agg(n_year=("year", "nunique")).sort_values(
    by="n_year"
)

,n_year
province,
A Coruña,4
Palencia,4
Cantabria,4
Burgos,4
Soria,4
...,...
Granada,25
Zaragoza,25
Huelva,25


Not all provinces have data for all years.

In [10]:
olive_yield_data.isna().sum()

province       0
total          0
table_olive    0
olive_oil      0
year           0
dtype: int64

# Validate province mapping

In [11]:
spain_map_path = Path("../data/processed/maps/")

In [12]:
spain_map = gpd.read_file(spain_map_path)

In [13]:
spain_map.head()

,community,province,geometry
0,País Vasco/Euskadi,Araba/Álava,"POLYGON ((-2.76808 42.61408, -2.76863 42.61334..."
1,Castilla-La Mancha,Albacete,"POLYGON ((-2.55212 38.08501, -2.55207 38.08517..."
2,Comunitat Valenciana,Alacant/Alicante,"MULTIPOLYGON (((-0.75223 37.88691, -0.75225 37..."
3,Andalucía,Almería,"MULTIPOLYGON (((-3.03624 35.93791, -3.03637 35..."
4,Castilla y León,Ávila,"POLYGON ((-5.43382 40.24491, -5.4343 40.24276,..."


In [14]:
def is_province(province_name: str) -> bool:
    return any(spain_map["province"] == province_name) or any(
        spain_map["community"] == province_name
    )

In [15]:
provinces_from_data = olive_yield_data["province"].unique()

In [16]:
for province in provinces_from_data:
    if not is_province(province):
        print(f"{province}")

ESPAÑA


Some rows in the dataset represents aggregations at the community level. We should remove these rows.

To do so, for each value in the `province` column we need to know whther it's a province or communtiy.


In [17]:
spain_provinces = spain_map["province"].unique()

In [18]:
single_province_communities = (
    spain_map["community"].value_counts().pipe(lambda x: x[x == 1]).index
).values

In [19]:
print(single_province_communities)

['Illes Balears' 'La Rioja' 'Comunidad de Madrid' 'Región de Murcia'
 'Comunidad Foral de Navarra' 'Principado de Asturias' 'Cantabria'
 'Ciudad Autónoma de Ceuta' 'Ciudad Autónoma de Melilla']


In [20]:
provinces_to_keep = np.unique(
    np.concatenate((single_province_communities, spain_provinces))
)

In [21]:
print(provinces_to_keep)

['A Coruña' 'Alacant/Alicante' 'Albacete' 'Almería' 'Araba/Álava'
 'Asturias' 'Badajoz' 'Barcelona' 'Bizkaia' 'Burgos' 'Cantabria'
 'Castelló/Castellón' 'Ceuta' 'Ciudad Autónoma de Ceuta'
 'Ciudad Autónoma de Melilla' 'Ciudad Real' 'Comunidad Foral de Navarra'
 'Comunidad de Madrid' 'Cuenca' 'Cáceres' 'Cádiz' 'Córdoba' 'Gipuzkoa'
 'Girona' 'Granada' 'Guadalajara' 'Huelva' 'Huesca' 'Illes Balears' 'Jaén'
 'La Rioja' 'Las Palmas' 'León' 'Lleida' 'Lugo' 'Madrid' 'Melilla'
 'Murcia' 'Málaga' 'Navarra' 'Ourense' 'Palencia' 'Pontevedra'
 'Principado de Asturias' 'Región de Murcia' 'Salamanca'
 'Santa Cruz de Tenerife' 'Segovia' 'Sevilla' 'Soria' 'Tarragona' 'Teruel'
 'Toledo' 'Valladolid' 'València/Valencia' 'Zamora' 'Zaragoza' 'Ávila']


In [22]:
olive_yield_data.shape

(1263, 5)

In [23]:
olive_yield_data = olive_yield_data[
    olive_yield_data["province"].isin(provinces_to_keep)
]

In [24]:
olive_yield_data.shape

(1000, 5)

# Fill in missing years:

Each province should have data for the whole years range, from `min_year` to `max_year`.

Added years will have zero values.

This is done to ensure the data is aligned in a way where each province have the same number of years.

In [25]:
min_year = olive_yield_data["year"].min()
max_year = olive_yield_data["year"].max()

In [26]:
print(f"Min year: {min_year}")
print(f"Max year: {max_year}")

Min year: 1998
Max year: 2022


In [27]:
years = pd.Series(range(min_year, max_year + 1), name="year")

In [28]:
provinces = pd.Series(olive_yield_data["province"].unique(), name="province")

In [29]:
all_provinces_years_df = pd.merge(left=years, right=provinces, how="cross")

In [30]:
all_provinces_years_df

,year,province
0,1998,Araba/Álava
1,1998,Navarra
2,1998,La Rioja
3,1998,Huesca
4,1998,Teruel
...,...,...
1245,2022,Cantabria
1246,2022,Bizkaia
1247,2022,Burgos
1248,2022,Palencia


In [31]:
olive_yield_data_processed = (
    pd.merge(
        left=all_provinces_years_df,
        right=olive_yield_data,
        on=["province", "year"],
        how="left",
    )
    .sort_values(by=["province", "year"])
    .fillna(0.0)
)

In [32]:
olive_yield_data_processed.head()

,year,province,total,table_olive,olive_oil
43,1998,A Coruña,0.0,0.0,0.0
93,1999,A Coruña,0.0,0.0,0.0
143,2000,A Coruña,0.0,0.0,0.0
193,2001,A Coruña,0.0,0.0,0.0
243,2002,A Coruña,0.0,0.0,0.0


In [33]:
olive_yield_data_processed.dtypes

year                     int64
province       string[pyarrow]
total                  float64
table_olive            float64
olive_oil              float64
dtype: object

Convert numerical columns to `float32` for space efficiency:

In [34]:
olive_yield_data_processed = olive_yield_data_processed.astype(
    dtype={"total": "float32", "table_olive": "float32", "olive_oil": "float32"}
)

In [35]:
output_path = Path("../data/processed/olive_yield/")

In [36]:
olive_yield_data_processed.to_parquet(output_path / "olive_yield.parquet", index=False)